In [2]:
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:36<00:00,  7.25s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
from datasets import load_dataset

from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor
import librosa
from datasets import load_dataset
import torch

# Replace 'your_dataset_name' with the name of your Hugging Face dataset
dataset = load_dataset("cairocode/IEMO_WAV_002")


import torch
import numpy as np
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
import librosa
import soundfile as sf
from io import BytesIO

# Define the emotion mapping
emotion_mapping = {
    "Neutral": 0,
    "Happy": 1,
    "Sad": 2,
    "Angry": 3,
}

In [4]:
from datasets import concatenate_datasets

# Define the filter function with an argument
def create_filter_function(spk_id):
    def filter_m_examples(example):
        return example["label"] != 4 and example["label"] != 5 and example["speakerID"] == spk_id
    return filter_m_examples

# Use the filter function with the specific speaker ID
spk_id = 2
filter_function = create_filter_function(spk_id)
ds_002 = dataset['train'].filter(filter_function)

In [5]:
ds = ds_002

In [17]:
actual_labels

[0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
from datasets import Dataset
from collections import defaultdict
import random

def group_by_label(dataset, label_key):
    """Group dataset samples by label."""
    label_groups = defaultdict(list)
    for sample in dataset:
        label = sample[label_key]
        label_groups[label].append(sample)
    return label_groups

def create_balanced_batches(dataset, label_key, batch_size):
    """Create balanced batches ensuring at least one sample per label."""
    label_groups = group_by_label(dataset, label_key)
    batches = []
    labels = list(label_groups.keys())
    
    while any(len(label_groups[label]) > 0 for label in labels):
        batch = []
        random.shuffle(labels)  # Shuffle labels for randomness
        for label in labels:
            if len(batch) < batch_size and len(label_groups[label]) > 0:
                batch.append(label_groups[label].pop(0))  # Take one sample
        
        # Fill the remaining spots in the batch
        remaining_samples = [
            sample for label in labels
            for sample in label_groups[label][:max(0, batch_size - len(batch))]
        ]
        random.shuffle(remaining_samples)  # Add randomness to the selection
        batch.extend(remaining_samples[:batch_size - len(batch)])
        
        batches.append(batch)
    
    return batches

def balanced_batches_as_datasets(dataset, label_key, batch_size):
    """Create balanced batches and return them as a list of Dataset objects."""
    batches = create_balanced_batches(dataset, label_key, batch_size)
    dataset_batches = [
        Dataset.from_dict({key: [sample[key] for sample in batch] for key in dataset.column_names})
        for batch in batches
    ]
    return dataset_batches

# Example usage
# Assuming `train_dataset` is a Hugging Face Dataset and labels are in the 'label' column
batch_size = 8
balanced_dataset_batches = balanced_batches_as_datasets(ds, 'label', batch_size)

# Combine batches back into a single Dataset if needed
final_dataset = Dataset.from_dict({
    key: [sample for batch in balanced_dataset_batches for sample in batch[key]]
    for key in ds.column_names
})

print(final_dataset)


Dataset({
    features: ['audio', 'label', 'valence', 'arousal', 'domination', 'arousal_norm', 'valence_norm', 'speakerID', 'utterance_id', 'transcript', 'speaker_id'],
    num_rows: 1340
})


In [31]:
final_dataset['label']

[1,
 3,
 0,
 2,
 3,
 1,
 2,
 0,
 2,
 1,
 3,
 0,
 3,
 3,
 2,
 2,
 3,
 2,
 1,
 0,
 3,
 1,
 1,
 2,
 2,
 1,
 0,
 3,
 3,
 3,
 0,
 2,
 3,
 0,
 2,
 1,
 3,
 1,
 0,
 1,
 2,
 0,
 1,
 3,
 1,
 0,
 2,
 2,
 1,
 0,
 3,
 2,
 2,
 1,
 0,
 1,
 0,
 1,
 2,
 3,
 0,
 3,
 2,
 0,
 3,
 0,
 1,
 2,
 2,
 1,
 0,
 2,
 3,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 1,
 2,
 0,
 3,
 3,
 2,
 2,
 0,
 0,
 2,
 3,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 3,
 3,
 0,
 0,
 3,
 2,
 3,
 0,
 1,
 3,
 2,
 1,
 1,
 0,
 2,
 3,
 1,
 3,
 3,
 0,
 1,
 0,
 1,
 2,
 3,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 3,
 0,
 1,
 0,
 3,
 2,
 0,
 3,
 1,
 0,
 3,
 3,
 2,
 2,
 1,
 3,
 0,
 3,
 2,
 0,
 1,
 2,
 1,
 0,
 3,
 3,
 0,
 1,
 3,
 2,
 1,
 0,
 3,
 3,
 0,
 2,
 1,
 3,
 1,
 2,
 0,
 0,
 3,
 2,
 1,
 3,
 2,
 0,
 1,
 1,
 2,
 1,
 0,
 3,
 1,
 2,
 0,
 0,
 2,
 3,
 1,
 1,
 0,
 3,
 2,
 0,
 2,
 1,
 0,
 2,
 0,
 3,
 1,
 1,
 0,
 3,
 0,
 3,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 3,
 1,
 3,
 3,
 3,
 0,
 0,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 0,
 3,
 2,
 1,
 1,
 0,
 3,
 0,
 3,
 2,
 0,
 1,
 1,
 3,
 1,
 2,
 3,
 0,


In [33]:
import numpy as np
import soundfile as sf
from io import BytesIO
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration
import librosa
import warnings
import torch
import csv
warnings.filterwarnings("ignore", message="It is strongly recommended to pass the `sampling_rate` argument")

# Map emotions to numbers
emotion_mapping = {
    "Neutral": 0,
    "Happy": 1,
    "Sad": 2,
    "Angry": 3
}
# Reverse mapping: from number to emotion
reverse_emotion_mapping = {v: k for k, v in emotion_mapping.items()}
ds = final_dataset
# Load processor and model
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
# model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")

csv_file = "emotion_results_all.csv"
fieldnames = ["Transcript", "Label", "Predicted Emotion", "Mapped Value", "speaker_id"]
with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

batch_size = 8  # Adjust batch size as needed
num_samples = len(ds)  # Replace `ds` with your dataset

for batch_start in range(0, num_samples, batch_size):
    batch_end = min(batch_start + batch_size, num_samples)
    batch_indices = range(batch_start, batch_end)
    print(batch_start, batch_end, batch_indices)


    transcripts = [ds['transcript'][i] for i in batch_indices]
    actual_labels = [ds['label'][i] for i in batch_indices]
    speaker_ids = [ds['speakerID'][i] for i in batch_indices]

        # Process conversations
    conversations = [
        [
            {
                "role": "user",
                "content": [
                    # Only add "The correct answer was ..." if it's not the first in the batch
                    *(
                        [{"type": "text", "text": f"The correct answer was {reverse_emotion_mapping[actual_labels[i - 1 - batch_start]]}, use this to understand this person more."}]
                        if i > batch_start and i>0
                        else []
                    ),
                    {"type": "audio", "array": ds['audio'][i]['array']},
                    {"type": "text", "text": "Is she happy, sad, angry or neutral? Answer in one word"}
                ]
            }
        ]
        for i in batch_indices
    ]
    # print(i)



    # Prepare text inputs
    text = [processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False) for conversation in conversations]

    # Prepare audio inputs
    audios = []
    for conversation in conversations:
        for message in conversation:
            if isinstance(message["content"], list):
                for ele in message["content"]:
                    if ele["type"] == "audio":
                        # Convert numpy array to WAV and load with librosa
                        audio_data = ele["array"]
                        with BytesIO() as buffer:
                            sf.write(buffer, audio_data, samplerate=16000, format="WAV")  # Assuming 16kHz input
                            buffer.seek(0)
                            y, _ = librosa.load(buffer, sr=processor.feature_extractor.sampling_rate)
                            audios.append(y.astype(np.float32))

    # Ensure the number of text and audio samples match
    if len(audios) != len(text):
        raise ValueError(f"Mismatch between audio ({len(audios)}) and text ({len(text)}) inputs.")
    # Prepare inputs
    inputs = processor(
        text=text,
        audios=audios,
        sampling_rate=16000,  # Explicitly set the sampling rate here
        return_tensors="pt",
        padding=True
    )

    # Adjust attention_mask for valid start tokens
    inputs["attention_mask"][:, 0] = 1

    # Handle special audio token alignment
    special_audio_token_mask = (inputs["input_ids"] == model.config.audio_token_index)

    # Ensure special_audio_token_mask is valid for audio processing
    if special_audio_token_mask.sum() == 0:
        audio_token_id = getattr(model.config, "audio_token_index", None)
        if audio_token_id is not None:
            inputs["input_ids"][:, 0] = audio_token_id

    # Ensure batch size matches across all tensors
    text_batch_size = inputs["input_ids"].shape[0]
    audio_batch_size = inputs["input_features"].shape[0]

    if text_batch_size != audio_batch_size:
        raise RuntimeError(
            f"Batch size mismatch: Text batch size ({text_batch_size}) "
            f"does not match Audio batch size ({audio_batch_size})."
        )

    # Move tensors to the appropriate device
    inputs = {key: value.to("cuda") for key, value in inputs.items()}

    # Generate response
    try:
        generate_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            input_features=inputs["input_features"],
            feature_attention_mask=inputs["feature_attention_mask"],
            max_length=256
        )

        # Post-process generation output
        generate_ids = generate_ids[:, inputs["input_ids"].size(1):]

        # Decode the responses
        responses = processor.batch_decode(
            generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        for i, response in enumerate(responses):
            predicted_emotion = response.strip().capitalize()
            mapped_value = emotion_mapping.get(predicted_emotion, -1)

            print(f" | Actual Label: {actual_labels[i]} | Predicted Emotion: {mapped_value} | Transcript: {transcripts[i]}")

            # Save results to CSV
            with open(csv_file, mode="a", newline="") as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writerow({
                    "Transcript": transcripts[i],
                    "Label": actual_labels[i],
                    "Predicted Emotion": predicted_emotion,
                    "Mapped Value": mapped_value,
                    "speaker_id": speaker_ids[i]
                })

    except RuntimeError as e:
        print("RuntimeError during generation:", str(e))
        print("Debugging info:")
        print(f"Input IDs Shape: {inputs['input_ids'].shape}")
        print(f"Feature Attention Mask Shape: {inputs['feature_attention_mask'].shape}")
        print(f"Input Features Shape: {inputs['input_features'].shape}")
        print(f"Attention Mask Shape: {inputs['attention_mask'].shape}")


0 8 range(0, 8)
 | Actual Label: 1 | Predicted Emotion: 0 | Transcript: Well Vegas was awesome.
 | Actual Label: 3 | Predicted Emotion: 1 | Transcript: That's out of control.
 | Actual Label: 0 | Predicted Emotion: 3 | Transcript: Excuse me.
 | Actual Label: 2 | Predicted Emotion: 0 | Transcript: Did you get the mail? So you saw my letter?
 | Actual Label: 3 | Predicted Emotion: 2 | Transcript: The bus, I'm riding the bus because I want to.
 | Actual Label: 1 | Predicted Emotion: 3 | Transcript: It was at the slot machines.
 | Actual Label: 2 | Predicted Emotion: 1 | Transcript: I don't know.  I put in that. request too. They didn't...
 | Actual Label: 0 | Predicted Emotion: 2 | Transcript: Clearly.  You know, do you have like a supervisor or something?
8 16 range(8, 16)
 | Actual Label: 2 | Predicted Emotion: 0 | Transcript: Yeah.  I know.
 | Actual Label: 1 | Predicted Emotion: 2 | Transcript: And, um, I got married.
 | Actual Label: 3 | Predicted Emotion: 1 | Transcript: I'm not fru

KeyboardInterrupt: 